In [19]:
from typing import Any, Dict, List, Optional,Set, Tuple 
import numpy as np
import pandas as pd
from random import sample,random
from math import floor
import matplotlib.pyplot as plt
!gdown --id 1xB6ewaR3HMKzhMJgIgvMWHNODifHDHUJ
!gdown --id 1OP0-auufRe8cS_S7KVtiG15XIUglILze
def generate_data(size : int = 100,coefs :int = 4) -> np.array:
    """
    Untuk Dummy saja
    Generate Random data < Continous >
    size = How many observations
    coefs = How many features / how many chromosome
    Returns:
        np array (x), (y)
    """
    coeffs : List[float] = [0.4, 0.3 ,0.2, -0.1]
    x : List[np.array] = [[np.random.normal() for _ in range(len(coeffs))] for _ in range(size) ]
    y : List[np.array] = [np.dot(i,coeffs) for i in x]
    
    return np.array(x), np.array(y)



class Population:
    
    def __init__(self,TotalSize,Totalgenome: int) -> None:
        self.Totalgenome: int = Totalgenome
        self.totalSize : int = TotalSize
        self.bestIndividuals : List =[]
        self.population = []
        self.fitnessed  : Dict[str,Any]

    def createIndividu(self) -> np.array:
        """
        Create Individu using Real Numbers

        Returns:
            List[int]: Real numbers sequence
        """
        return np.random.normal(size=self.Totalgenome)
    
    def createPopulation(self) -> None :
        """
        Create Population with x size Depends on total size
        and append it to population list 
        
        Returns:
            None 
        """
        self.population.append([self.createIndividu() for _ in range(self.totalSize)])
   
    def fitness(self, individual , inputs : np.array, yTrue : np.array) -> Dict[str,Any]:
        """
        
        Fitness current Individual

        Args:
            individual (List[int]) --> Chromosome on a population
            inputs (np.array) ->> Input data
            yTrue (np.array) --> Groud truth
        Returns:
            Dict[str,Any]: [description]
        """
        info : Dict[str,Any] = {}
        bias : np.float = 0
        predicted : np.array =  bias + np.dot(inputs,individual) 
        yTrue_mean : np.float = np.mean(yTrue) 

        SST : np.float = np.sum(np.array([(y - yTrue_mean) ** 2 for y in yTrue]),axis=None)
        SSR : np.float = np.sum(np.array([(ytrue - ypred) ** 2 for ytrue,ypred in zip(yTrue,predicted)]),axis = None)
    
        bias :np.float = np.mean(np.sqrt(np.array([(ytrue - ypred) ** 2 for ytrue,ypred in zip(yTrue,predicted)])))
        RMSE : np.float = np.sqrt(SSR/ len(y))
        Rsquared : np.float = (1 - (SSR / SST))

        SSE : np.float = SSR / len(y)
        oneDivError : np.float = 1 / SSE

        info["Rsquared"] = Rsquared
        info["coeff"] = individual
        info["error"] = SSE
        info["RMSE"] = RMSE 
        info["divByOne"] = oneDivError
        info["bias"] = bias 
        self.fitnessed = info
        return info
    
    def evaluate_population(self, x:np.array ,y:np.array ,selectionSize : int) -> None :
        """
        Function to evaluate the best individual from current population

        Args:
           x (np.array) , Input data
           y (np.array, Ground truth
           selectionSize (int) : How many best Individuals you want to grab from the population

        Returns:
            None
        """
        fitness_list= [self.fitness(individual,x,y) for individual in self.population[0]]
        error_list : List[Dict[str,Any]] = sorted(fitness_list,key=lambda i : i["error"])
        best_individuals = error_list[: selectionSize]
        self.bestIndividuals.clear()
        self.bestIndividuals.append(best_individuals)
        
        print(f"Error {best_individuals[0]['error']}\n RSquared {best_individuals[0]['Rsquared']}\n") 


    def mutate(self,individual : List[float], probabilityMutating : float) -> List[float]:
        """
        
        Ini Fungsinya lakuin mutasi 

        Args:
            individual (List[float]): Individu pada populasi
            probabilityMutating (float): Probabilitas Individu saat ini bermutasi 

        Returns:
            List[float]: Individu Yang telah bermutasi < Jika dilakukan >
        """
        indx : List[int] = [i for i in range(len(individual))]

        totalMutatedGens : int = int(probabilityMutating * len(individual))
        indx_toMutate : List[int] = sample(indx,k = totalMutatedGens)
        for ind in indx_toMutate:
            choice : np.int = np.random.choice([-1,1])
            gene_transform : float = choice*random()

            individual[ind] = individual[ind] + gene_transform
        return individual
        
    def crossover(self, parent1 : Dict[str,Any], parent2 : Dict[str,Any]) -> None:
        """
           CrossOver / persilangan untuk menghasilkan keturunan 
           metode yang digunakan adalah Simple Arithmetic crossover
        Args:
            parent1 (Dict[str,Any]): [description]
            parent2 (Dict[str,Any]): [description]
        Return: None
        """
        
        anak_keturunan : Dict[int,Any] = {}
        index : List[int] = [i for i in range( self.Totalgenome )]
        indexRandomize : List[int] = sample(index, floor(0.5 * self.Totalgenome))
        IndexNotInRandomize : List[int] = [i for i in index if i not in indexRandomize]

        getCromosomeFromParent1 : List[Any] = [[i,parent1['coeff'][i]] for i in indexRandomize]
        getCromosomeFromParent2 : List[Any] = [[i,parent2["coeff"][i]] for i in IndexNotInRandomize]
        
        anak_keturunan.update({key :value for (key,value) in getCromosomeFromParent1})
        anak_keturunan.update({key : value for(key,value) in getCromosomeFromParent2})
    
        return [anak_keturunan[i] for i in index]

    def create_new_generation(self,probabilityMutating : float,probability_indiv_mutating:float) -> None:
        """
        Ini method untuk pergantian generasi Menggunakan generational model
        Create new population using the best individuals < elitisime >, self.bestIndividuals adalah
        Best individual
        

        Args:
            probabilityMutating(float) : 
            probability_indiv_mutating (float) : 

        Return:
            None
        """
        pasangan_sah = [sample(self.bestIndividuals[0],2) for _ in range( self.totalSize)]
        crossOverered_parents = [self.crossover(pasangan[0],pasangan[1]) for pasangan in pasangan_sah]
        pasangan_sah_indx = [i for i in range(self.totalSize)]
        pasanganCalonMutasi = sample(pasangan_sah_indx,floor(probabilityMutating * self.totalSize))
        PasanganMutasi = [[i,self.mutate(crossOverered_parents[i],probability_indiv_mutating)] for i in pasanganCalonMutasi]
        for anakMutasi in PasanganMutasi:
            crossOverered_parents[anakMutasi[0]] = anakMutasi[1]
        self.population.clear()
        self.population.append(crossOverered_parents)
    
if __name__ == "__main__":
    x,y = generate_data(100)
    np.random.seed(123) 
    pop =Population(100,4)
    pop.createPopulation()
    selectionSize = floor(0.1 * 100)
    max_generations = 50

    probability_crosss_over = 0.8
    probability_indiv_mutation = 0.25

    for i in range(3) :
        print(f"generation : {i}")
        pop.evaluate_population(x,y,10)
        pop.create_new_generation(probability_crosss_over,probability_indiv_mutation)
        
    
    print("Nilai x : ",x[0])
    print("Nilai Y : ",y[0])
    
    result = pop.bestIndividuals[0][0]['bias'] + np.dot(x[0],pop.bestIndividuals[0][0]["coeff"])
    print("Predicted = ",result)
    print(pop.bestIndividuals[0][0])
    

    #parent1 = pop.fitness(individual,x,y)
    #parent2 = pop.fitness(individual,x,y)

'gdown' is not recognized as an internal or external command,
operable program or batch file.


generation : 0
Error 0.5418681723217632
 RSquared -1.040468815335096

generation : 1
Error 0.017405290077613076
 RSquared 0.9344583176512129

generation : 2
Error 0.009535814619661406
 RSquared 0.9640917600366429

Nilai x :  [-0.03612962 -2.56864261  1.16333069  0.73527533]
Nilai Y :  -0.6259060248834009
Predicted =  -0.38784880053894666
{'Rsquared': 0.9640917600366429, 'coeff': [0.43452905192470437, 0.24537940725969198, 0.1771923291989879, -0.02996800704754226], 'error': 0.009535814619661406, 'RMSE': 0.09765149573693895, 'divByOne': 104.8678104477987, 'bias': 0.0740440307617023}


'gdown' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
def terminate(ans : Dict[str,Any]):
    """
    Terminate Genetic Algorithm Menggunakan nilai RSquared yang telah didefinisiskan
    
    ans : Dict - > Berisi nilai Rsquared,coeff,RMSE ...
    
    return 
    True jika Rsquared > 0.95
    False others
    """
    if ans["Rsquared"] > 0.95:
        return True
    else :
        return False

In [21]:
from sklearn.preprocessing import StandardScaler
StdScaler = StandardScaler()


In [17]:
"""
Codingan Ini digunakan untuk preprocessing data saham dari link berikut : https://github.com/wildangunawan/Dataset-Saham-IDX
menjadi nilai open selama 10 hari berturut turut
"""
df = pd.read_csv("saham.csv")
dfnew = pd.DataFrame()
yow = []
def generate_newdataset():
    k=13
    j=1
    for i in range(100):
        
        dict1 = {}
        dict1.update( {i:df[j:k]["open_price"].values})
        j+=1
        k+=1
        yow.append(dict1)
generate_newdataset()
dfnew = pd.DataFrame(yow[0])
for i in range(1,len(yow)):
    dfnew[i] = pd.DataFrame(yow[i])
dfnew = dfnew.T
dfnew.to_csv("DatasetRequired.csv")


In [26]:
if __name__ == "__main__":
    df = pd.read_csv("DatasetRequired.csv")
    df = df.iloc[:,1:13]
    x = df.iloc[:,1:12]
    y = df.iloc[:,11]
    
    y = StdScaler.fit_transform(np.array(y).reshape(-1,1))
    x = StdScaler.fit_transform(x) 
    np.random.seed(123) 
    pop =Population(100,11)
    pop.createPopulation()
    selectionSize = floor(0.1 * 100)
    max_generations = 50

    probability_crosss_over = 0.95
    probability_indiv_mutation = 0.1
    

    terminate_ga = False
    generation = 0
    while(terminate_ga != True): 
        print(f"generation : {generation}")
        pop.evaluate_population(x,y,10)
        pop.create_new_generation(probability_crosss_over,probability_indiv_mutation)
        terminate_ga = terminate(pop.fitnessed)
        generation += 1
        
    print("Nilai x : ",x[0])
    print("Nilai Y : ",y[0])
    
    result = pop.bestIndividuals[0][0]['bias'] + np.dot(x[0],pop.bestIndividuals[0][0]["coeff"])
    print("Predicted = ",result)
    print(pop.bestIndividuals[0][0])

generation : 0
Error 1.0696958324096997
 RSquared -0.06969583240969945

generation : 1
Error 0.5056962777707011
 RSquared 0.49430372222929897

generation : 2
Error 0.3749903498761506
 RSquared 0.6250096501238496

generation : 3
Error 0.12210630390859731
 RSquared 0.8778936960914028

generation : 4
Error 0.10153619468371065
 RSquared 0.8984638053162893

generation : 5
Error 0.10763348026409857
 RSquared 0.8923665197359014

generation : 6
Error 0.04885598618448743
 RSquared 0.9511440138155126

generation : 7
Error 0.07083691577266142
 RSquared 0.9291630842273386

generation : 8
Error 0.052088883946247465
 RSquared 0.9479111160537526

generation : 9
Error 0.07379505748453909
 RSquared 0.9262049425154609

generation : 10
Error 0.06613880693565415
 RSquared 0.9338611930643459

generation : 11
Error 0.062268916479069236
 RSquared 0.9377310835209308

generation : 12
Error 0.05219604881168019
 RSquared 0.9478039511883198

generation : 13
Error 0.044498454645192434
 RSquared 0.9555015453548076


In [27]:

predicted = pop.bestIndividuals[0][0]['bias'] + np.dot(x, pop.bestIndividuals[0][0]["coeff"])
predicted

array([ 1.47517035,  1.32509922,  0.94080963,  1.09234881,  1.34848838,
        1.07104381,  0.76980561,  0.70645798,  0.43603377, -0.12153443,
       -0.13060267, -0.01278899,  0.24661104,  0.49387535,  0.54582486,
        0.11906527, -0.28392839, -0.10064011,  0.07900175, -0.25070277,
       -0.33034449,  0.45613953,  0.96877616,  0.17455368, -0.28146625,
       -0.18933982,  0.57954015,  0.29507866, -0.0802017 , -0.13470212,
       -0.20021621, -0.10054788, -0.24709062, -0.36941219, -0.35331633,
       -0.54135682, -1.1832694 , -2.09278076, -1.9767066 , -2.10170037,
       -2.00732842, -1.96987143, -1.86689041, -1.16753477, -1.44386122,
       -1.54705288, -1.14026775, -1.27055805, -0.73979465, -0.53221858,
       -0.62752151,  0.17385109,  0.75809414,  0.41807233,  0.07198529,
        0.14417001,  0.15010136,  0.10920526, -0.26384078, -0.30730247,
        0.54470317,  0.01497156, -0.09566306, -0.03620579,  0.04731072,
       -0.15016888, -0.28825907, -0.70555813, -0.10731423, -0.05

In [24]:
%whos

Variable                     Type                          Data/Info
--------------------------------------------------------------------
Any                          _Any                          typing.Any
Dict                         GenericMeta                   typing.Dict
List                         GenericMeta                   typing.List
Optional                     _Optional                     typing.Optional
Population                   type                          <class '__main__.Population'>
Set                          GenericMeta                   typing.Set
StandardScaler               type                          <class 'sklearn.preproces<...>ng._data.StandardScaler'>
StdScaler                    StandardScaler                StandardScaler()
Tuple                        TupleMeta                     typing.Tuple
df                           DataFrame                              0       1       <...>\n[100 rows x 12 columns]
dfnew                        DataFrame